**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.6MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")
for s,p,o in g:
  print(s,p,o)

http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#UPM http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#University
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [133]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#");
q1 = prepareQuery('''
  SELECT 
    ?x 
	WHERE { 
    ?x rdfs:subClassOf/(rdfs:subClassOf*) ns:Person
  } 
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)

print("Sparql\n")

for r in g.query(q1):
  print(r.x)

print("\nRDFLIB\n")
# llamada recursiva hasta que no haya mas subclases
def subclass_recursive(n):
  x = g.triples((None, RDFS.subClassOf, n))
  for s,p,o in x:
    print(s)
    return subclass_recursive(s)
  return

subclass_recursive(ns.Person)

Sparql

http://somewhere#Researcher
http://somewhere#PhDstudent

RDFLIB

http://somewhere#Researcher
http://somewhere#PhDstudent


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [169]:
q2 = prepareQuery('''
  SELECT 
    DISTINCT ?y
	WHERE { 
    OPTIONAL { ?x rdfs:subClassOf/(rdfs:subClassOf*) ns:Person . }
    ?y a ?z . FILTER (?z = ns:Person || ?z = ?x) 
  } 
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)

print("Sparql\n")

for r in g.query(q2):
  print(r.y)


print("\nRDFLIB\n")
# llamada recursiva para recoger todas las sublases, use a set to ensure there are not duplicates
def get_class_subclass_recursive(n,res):
  x = g.triples((None, RDFS.subClassOf, n))
  for s,p,o in x:
    # ensure we dont get in a loop
    if(s in res):
      return res
    res.add(s)
    return get_class_subclass_recursive(s, res)
  return res

ls = get_class_subclass_recursive(ns.Person, set())
ls.add(ns.Person);
setOfPersons = set()
for x in ls:
  y = g.triples((None, RDF.type, x))
  for s,p,o in y:
    # use a set to ensure there are not duplicates
    setOfPersons.add(s)

for x in setOfPersons:
  print(x)

Sparql

http://somewhere#SaraJones
http://somewhere#JaneSmith
http://somewhere#JohnSmith

RDFLIB

http://somewhere#JaneSmith
http://somewhere#JohnSmith
http://somewhere#SaraJones


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [172]:
 q3 = prepareQuery('''
  SELECT 
    DISTINCT ?y ?p ?o
	WHERE { 
    OPTIONAL { ?x rdfs:subClassOf/(rdfs:subClassOf*) ns:Person . }
    ?y a ?z . FILTER (?z = ns:Person || ?z = ?x)
    ?y ?p ?o .
  } 
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)
 
print("Sparql\n")

for r in g.query(q3):
  print(r.y, r.p, r.o)

print("\nRDFLIB\n")

for x in setOfPersons:
  for s,p,o in g.triples((x,None,None)):
    print(s,p,o)


Sparql

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somew